In [93]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns

# # Read data into a dataframe and 
# df = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = df.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]
# densityPlot = sns.kdeplot(counts, bw_adjust=3)
# yLabels = densityPlot.get_yticks()
# densityPlot.set_yticklabels('{:,.0%}'.format(y) for y in yLabels)
# plt.xlabel("Number of reviews")
# plt.ylabel("Percentage of users")
# plt.savefig(fname="densityPlot")
# plt.show()

In [94]:
# import pandas as pd

# # Read ratings into a dataframe and narrow down reviewers to desired review count
# ratings = pd.read_csv('ratings.csv', dtype={"vote": str})
# counts = ratings.groupby('reviewerID')['reviewerID'].size()
# counts = counts[(counts >=5) & (counts <= 10)]

# # Create and apply mask that removes all other reviewers.
# mask = ratings['reviewerID'].isin(counts.index)
# ratings = ratings[mask]

# # Read metadata into dataframe and define asins that are used by making a set
# metadata = pd.read_csv("metadata.csv")
# used_asins = set(ratings["asin"])

# # Create and apply mask that removes unreferenced items
# mask = metadata["asin"].isin(used_asins)
# metadata = metadata[mask]
# metadata.to_pickle("metadata.pickle")
# ratings.to_pickle("ratings.pickle")


In [95]:
# import pandas as pd
# import numpy as np
# from scipy.sparse import csr_matrix, save_npz
# ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
# metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
# ratings = ratings[["reviewerID", "asin", "overall"]]

# # Series of each unique value of reviewerID and asin 
# reviewerIDs = ratings['reviewerID'].unique()
# asins = ratings["asin"].unique()
# # Dictionary that maps reviewers to an index
# reviewerMap = {reviewerID: i for i, reviewerID in enumerate(reviewerIDs)}

# # Dictionary that maps asins to an index
# asinMap = {asin: i for i, asin in enumerate(asins)}

# # Initialize the utility matrix with zeros
# utilityMatrixArray = np.zeros((len(reviewerIDs), len(asins)), "int8")

# # Fill in the utility matrix with data from the records array
# for row in ratings.itertuples():
#     reviewerIndex = reviewerMap[row.reviewerID]
#     asinIndex = asinMap[row.asin]
#     utilityMatrixArray[reviewerIndex, asinIndex] = row.overall
# utilityMatrix = csr_matrix(utilityMatrixArray)
# save_npz("utilityMatrix.npz", utilityMatrix)

In [96]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import sqrt
from scipy.sparse import load_npz
from sklearn.decomposition import TruncatedSVD

ratings : pd.DataFrame = pd.read_pickle("ratings.pickle")
metadata : pd.DataFrame = pd.read_pickle("metadata.pickle")
utilityMatrix = load_npz("utilityMatrix.npz")

# Create training and testing datasets
trainingData = utilityMatrix[:113404]
testingData = utilityMatrix[113404:]

# X_train, X_test= train_test_split(utilityMatrix, test_size=0.2, random_state=42)
model = TruncatedSVD(n_components=50, random_state=42).fit(trainingData)
resultant_matrix = model.transform(trainingData)
var_explained = model.explained_variance_ratio_.sum()
var_explained
# densityPlot = sns.kdeplot(, y=svd.explained_variance_ratio_, bw_adjust=1)
# plt.show()

0.20602141827315013